In [2]:
%pip install confluent-kafka
%pip install pyspark
%pip install pymongo
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, month, year
from pyspark.sql.functions import split, col, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import confluent_kafka as ck
import random
import time
import requests
import csv
from io import StringIO
import pandas as pd
from datetime import datetime, timedelta, date
import calendar
import re
import time

# Kafka

## Config Kafka

In [2]:
config = {}
# Required connection configs for Kafka producer, consumer, and admin
config["bootstrap.servers"]="pkc-60py3.europe-west9.gcp.confluent.cloud:9092"
config["security.protocol"]="SASL_SSL"
config["sasl.mechanisms"]="PLAIN"
config["sasl.username"]="GDHUJVNTILOC36TK"
config["sasl.password"]="sQ4OhM7DZbJrDx4j00B6IUuxRG8cQmjM1fSZg/foCiKD1RHCCU1DYIGCj+ouY02O"

# Best practice for higher availability in librdkafka clients prior to 1.7
config["session.timeout.ms"]=45000

config["group.id"] = "python-group-1"
config["auto.offset.reset"] = "earliest"

"""
# Required connection configs for Confluent Cloud Schema Registry
schema.registry.url="https://psrc-9jzo5.us-central1.gcp.confluent.cloud"
basic.auth.credentials.source="USER_INFO"
basic.auth.user.info="V62ZHMWOWECRACCJ:no7BCD6mPA85DsECXfSKijlOJpHDAYcemUiIZAjk7d49AUuEwdJKXCquZZm4atTN"
"""


'\n# Required connection configs for Confluent Cloud Schema Registry\nschema.registry.url="https://psrc-9jzo5.us-central1.gcp.confluent.cloud"\nbasic.auth.credentials.source="USER_INFO"\nbasic.auth.user.info="V62ZHMWOWECRACCJ:no7BCD6mPA85DsECXfSKijlOJpHDAYcemUiIZAjk7d49AUuEwdJKXCquZZm4atTN"\n'

# Consumer

In [23]:
from io import StringIO
consumer = ck.Consumer(config)
consumer.subscribe(["meteo_data"])
try:
     while True:
          msg = consumer.poll(1.0)
          if msg is not None and msg.error() is None:
               df = pd.DataFrame(columns=["station_id", "dh_utc", "temperature", "vent_moyen", "pluie_1h"])
               input_strings = msg.value().decode('utf-8').split('\n')
               for input_string in input_strings:
                    new_data = pd.read_csv(StringIO(input_string), sep=';', names=df.columns)
                    df = pd.concat([df, new_data], ignore_index=True)
               calc_moy(df)
               #display(df)
except KeyboardInterrupt:
     pass
finally:
     consumer.close()


C:\Users\Julian\AppData\Local\Temp\ipykernel_14748\3747825510.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


,station_id,year,month,avg_temperature,avg_vent_moyen,avg_pluie_1h
0,23,2008,4,8.501081,10.497296,0.351351


C:\Users\Julian\AppData\Local\Temp\ipykernel_14748\3747825510.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


,station_id,year,month,avg_temperature,avg_vent_moyen,avg_pluie_1h
0,23,2008,1,6.170833,8.268516,0.214815


C:\Users\Julian\AppData\Local\Temp\ipykernel_14748\3747825510.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


...

,station_id,year,month,avg_temperature,avg_vent_moyen,avg_pluie_1h
0,42,2023,11,8.947859,5.982875,0.164832
1,000BZ,2023,11,10.045646,4.603371,0.056461
2,000CJ,2023,11,2.484158,2.097313,0.405092
3,000DF,2023,11,11.553683,14.648300,0.246459
4,000DR,2023,11,8.075499,1.463533,0.279772
5,000EB,2023,11,8.746056,8.948310,0.159437
6,000EU,2023,11,8.002244,8.453156,0.116690
7,000FA,2023,11,7.420280,7.890769,0.191122
8,000FN,2023,11,9.301458,5.441837,0.164431
9,000HL,2023,11,11.351473,3.843759,0.068163


C:\Users\Julian\AppData\Local\Temp\ipykernel_14748\3747825510.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


,station_id,year,month,avg_temperature,avg_vent_moyen,avg_pluie_1h
0,42,2023,12,7.560976,6.439431,0.098916
1,000BZ,2023,12,7.541723,4.310363,0.055451
2,000CJ,2023,12,2.132157,1.187381,0.274084
3,000DF,2023,12,9.320439,12.457064,0.130041
4,000DR,2023,12,6.355163,1.200815,0.162772
5,000EB,2023,12,7.066985,9.268213,0.095498
6,000EU,2023,12,6.097035,8.047844,0.077898
7,000FA,2023,12,6.276216,9.409459,0.136954
8,000FN,2023,12,8.426389,7.341111,0.136389
9,000HL,2023,12,9.033018,2.485250,0.081732


C:\Users\Julian\AppData\Local\Temp\ipykernel_14748\3747825510.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


,station_id,year,month,avg_temperature,avg_vent_moyen,avg_pluie_1h
0,42,2024,1,3.882055,5.847471,0.079608
1,000BZ,2024,1,6.328237,5.712480,0.043682
2,000CJ,2024,1,0.642812,0.714377,0.235463
3,000DF,2024,1,6.964184,12.293978,0.058320
4,000DR,2024,1,4.381760,1.057760,0.099200
5,000EB,2024,1,3.777456,7.797746,0.140419
6,000EU,2024,1,3.869836,8.742295,0.045902
7,000FA,2024,1,2.412461,8.957632,0.121461
8,000FN,2024,1,4.713562,5.955392,0.243137
9,000HL,2024,1,8.436562,3.127188,0.115937


In [9]:
display(df)

,station_id,dh_utc,temperature,vent_moyen,pluie_1h
0,23,2019-11-01 00:00:00,9.2,5.0,0.0
1,23,2019-11-01 01:00:00,9.3,6.0,0.0
2,23,2019-11-01 02:00:00,9.3,5.0,0.0
3,23,2019-11-01 03:00:00,9.3,2.0,0.0
4,23,2019-11-01 04:00:00,9.3,5.0,0.0
...,...,...,...,...,...
17126,ME014,2019-11-29 14:00:00,8.6,0.0,0.0
17127,ME014,2019-11-29 15:00:00,8.3,0.0,0.0
17128,ME014,2019-11-29 16:00:00,7.9,0.0,0.0
17129,ME014,2019-11-29 19:00:00,6.7,0.0,0.0


# Creation Session Spark

In [3]:
spark = SparkSession.builder.master("local[*]").appName("YourAppName").config("spark.driver.memory", "4g").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").config("spark.executor.cores", "4").getOrCreate() 

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: The value of spark.network.timeout=15000ms must be greater than the value of spark.executor.heartbeatInterval=15000ms.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.SparkConf.validateSettings(SparkConf.scala:571)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:410)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(Unknown Source)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(Unknown Source)
	at java.lang.reflect.Constructor.newInstance(Unknown Source)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.j

# Calcul des valeurs moyennes sur le mois

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

def calcul_vals_moyennes(df):

    # Convert pandas DataFrame to Spark DataFrame
    data_df = spark.createDataFrame(df, schema=["station_id", "dh_utc", "temperature", "vent_moyen", "pluie_1h"])

    # Extraction du mois et de l'année à partir de la colonne dh_utc
    data_df = data_df.withColumn("month", month("dh_utc"))
    data_df = data_df.withColumn("year", year("dh_utc"))

    stations_csv_path = "./stations_formatted.csv"

    # Define the schema explicitly
    schema = StructType([
        StructField("id_station", StringType(), True),
        StructField("nom_station", StringType(), True)
    ])

    # Charger le fichier de correspondance dans un DataFrame Spark
    stations_df = spark.read.option("delimiter", ":").csv(stations_csv_path, header=True, schema=schema)
    # Supprimer la ligne contenant "metadata null"
    # stations_df = stations_df.filter(~col("id_station").contains("metadata") & ~col("nom_station").contains("null"))

    # Jointure entre df_filtered et stations_df
    df_joined = data_df.join(stations_df, data_df.station_id == stations_df.id_station)


    # Grouper les données par station_id, nom_station et mois, puis calculer les moyennes
    average_df = df_joined.groupBy("station_id").agg(
        avg("pluie_1h").alias("avg_pluie_1h"),
        avg("vent_moyen").alias("avg_vent_moyen"),
        avg("temperature").alias("avg_temperature")
    )

    # On retourne les résultats
    return(average_df)
    

test = calcul_vals_moyennes(df)
test.show()

Py4JJavaError: An error occurred while calling o164.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 8) (DESKTOP-IQL3AGM executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoi

# Calcul moyennes sur 1 mois SANS SPARK + sauvegarde d'un mois en CSV

In [16]:
def calc_moy(df):
    # Group by 'station_id' and calculate overall averages
    df['dh_utc'] = pd.to_datetime(df['dh_utc'])
    df['year'] = df['dh_utc'].dt.year
    df['month'] = df['dh_utc'].dt.month
    average_df = df.groupby(['station_id', 'year', 'month']).agg(
        avg_temperature=pd.NamedAgg(column='temperature', aggfunc='mean'),
        avg_vent_moyen=pd.NamedAgg(column='vent_moyen', aggfunc='mean'),
        avg_pluie_1h=pd.NamedAgg(column='pluie_1h', aggfunc='mean')
    ).reset_index()

    display(average_df)
    csv_file_name = f"./data/{average_df['month'].iloc[0]}_{average_df['year'].iloc[0]}.csv"
    average_df.to_csv(csv_file_name, index=False)
